In [1]:
# Déterminer le gène d'intérêt
gene = 'ENSG00000012048'

In [2]:
##### ÉTAPE 1: FORMER LE DICTIONNAIRE GENE_DIC POUR LE GÈNE D'INTÉRÊT #####
# Le gene_dic contient tous les transcrits d'un gène d'intérêt, avec tous les exons contenus dans les transcrit et la position génomique de
# départ et de fin de chaque exon. 

import re # Importation du module pour les regular expression
gene_dic = {} # Création du dictionnaire vierge
gene_dic[gene]= {} # Ajoute le nom du gène (identifiant ENSG) comme 1ere clé et un sous-dictionnaire comme valeur
data = 'Homo_sapiens.GRCh38.113.gtf'
with open(data, 'r') as file: # Ouverture du GTF en mode read()
    for line in file: # Itération sur chaque ligne du GTF

        # Séparation dans les lignes à l'aide des caractères '\t' et '"'
        colonne_tab = line.split('\t') 
        colonne_guillemet = line.split('"')
            
        if gene in line: # Conserve les lignes qui portent sur le gène d'intérêt dans le GTF

            if colonne_tab[2]== 'gene': # Retient la ligne sur le gène général
                brand = colonne_tab[6] # Stock dans une variable la nature du brin +/-
                gene_name = colonne_guillemet[5] # Stock dans une variable le nom commun du gène. Ex. BRCA1

            if colonne_tab[2]== 'transcript': # Retient les lignes sur les transcrits
                transcrit_nom = colonne_guillemet[5] # Stock le nom du transcrit ENST dans une variable
                gene_dic[gene][transcrit_nom]= {} # Ajoute le nom des transcrits au 1er sous-dictionnaire (clé) et un sous-dictionnaire comme valeur

            if colonne_tab[2]== 'exon': # Retient les lignes d'exons
                transcrit_name_info = re.search(r'ENST[0123456789]{5,100}', line) # Recherche le nom du transcrit dans la ligne de l'exon
                transcrit_name = transcrit_name_info.group(0) # Stock le nom du transcrit ENST dans une variable
                exon_name_info = re.search(r'ENSE[0123456789]{5,100}', line) # Recherche le nom de l'exon dans la ligne de l'exon
                exon_name = exon_name_info.group(0) # Stock le nom de l'exon ENSE dans une variable
                petit = colonne_tab[3] # Position génomique la plus petite
                grand = colonne_tab[4] # Position génomique la plus grande

                for key in gene_dic[gene].keys(): # Itération sur chaque transcrit du gène
                    if key == transcrit_name: # La clé (transcrit) doit être le même que le nom du transcrit de la ligne
                        gene_dic[gene][transcrit_name][exon_name]= [] # Ajoute tous les exons appartenant au transcrit (clé) et une liste vide comme valeur

                for key in gene_dic[gene][transcrit_name].keys(): # Itération sur chaque exon présent dans un transcrit
                    if key == exon_name: # La clé (exon) doit être le même que le nom de l'exon de la ligne
                        gene_dic[gene][transcrit_name][exon_name].append(petit) # Ajoute la première position génomique dans la liste
                        gene_dic[gene][transcrit_name][exon_name].append(grand) # Ajoute la deuxième position génomique dans la liste

print(gene_dic)

{'ENSG00000012048': {'ENST00000497488': {'ENSE00001867485': ['43125271', '43125300'], 'ENSE00003786858': ['43091435', '43094860'], 'ENSE00004011558': ['43090944', '43091032'], 'ENSE00004011560': ['43082404', '43082575'], 'ENSE00004011561': ['43076488', '43076614'], 'ENSE00004011562': ['43074331', '43074521'], 'ENSE00004011554': ['43070928', '43071238'], 'ENSE00004011556': ['43067608', '43067695'], 'ENSE00004011565': ['43063874', '43063951'], 'ENSE00004011567': ['43063333', '43063373'], 'ENSE00004011551': ['43057052', '43057135'], 'ENSE00004011553': ['43051063', '43051117'], 'ENSE00004011568': ['43049121', '43049194'], 'ENSE00004011552': ['43047643', '43047703'], 'ENSE00004011564': ['43044295', '43045802']}, 'ENST00000489037': {'ENSE00001930573': ['43125182', '43125321'], 'ENSE00003559512': ['43124017', '43124115'], 'ENSE00003510592': ['43115726', '43115779'], 'ENSE00003531836': ['43104868', '43104956'], 'ENSE00001917948': ['43104122', '43104261'], 'ENSE00004011563': ['43099775', '43099

In [3]:
##### ÉTAPE 2: CRÉER LA FONCTION INTERNE SEQUENCE_cDNA() #####
# La fonction sequence_cDNA() permet de sortir la séquence d'ADNc de n'importe quel transcrit d'un gène 

def sequence_cDNA (transcrit): # Définir la fonction; elle dépend seulement du transcrit
    header_of_transcrit = '' # Création d'une chaine qui contiendra le header recherché

    with open('Homo_sapiens.GRCh38.cdna.all.fa', 'r') as file: # Ouverture du fasta qui contient la séquence ADNc de tous les transcrits des gènes Homo sapiens
        for line in file: # Itération sur chaque ligne du fichier
            if gene in line and transcrit in line: # Retient la ligne avec le nom du gène et le nom du transcrit
                header_of_transcrit = line.strip() # Cette ligne devient le header_of_transcrit, car elle contient le gène et le transcrit recherché
                break # Quand le header est trouvé, on sort de la boucle

    if header_of_transcrit: # Si un header a été trouvé
        sequence = "" # Création d'une chaine qui contiendra la séquence ADNc 
        header_trouve = False

        with open('Homo_sapiens.GRCh38.cdna.all.fa', 'r') as file: # Ouverture du fichier fasta qui contient les séquences d'ADNc
            for line in file: # Itération sur chaque ligne du fichier
                line = line.strip()
                if line.startswith(">"): # Retient les lignes commençant par '>', donc les headers
                    if header_trouve:
                        break

                    if line == header_of_transcrit: # La ligne correspond au header recherché
                        header_trouve = True

                elif header_trouve:
                    sequence += line

        if sequence: # Si une séquence a été extraite
            return sequence
            
        else: # Si aucune n'a été trouvé
            return ("Séquence non trouvée.")
            
    else: # Si aucun header n'a été trouvé
        return (f"Le header {header_of_transcrit} n'a pas été trouvé.")


In [4]:
##### ÉTAPE 3: CRÉER LA FONCTION INTERNE GET_POSITION_CHRONOLOGIQUE() #####
# La fonction get_position_chronologique() a pour but de convertir n'importe quelle position génomique en position chronologique dans un
# transcrit spécifique d'un gène spécifique. La position chronologique retournée commence par 1.

def get_position_chronologique(position_chromosomique, transcrit): # Définir la fonction; elle dépend du transcrit et de la position chromosomique
    total = 0
    liste_position_fin_exon = [0] # Liste qui va contenir la position de fin de chaque exon en coordonnées chronologiques

    for exon in gene_dic[gene][transcrit]: # Itération sur chaque exon du transcrit
        longueur_exon = int(gene_dic[gene][transcrit][exon][1])-int(gene_dic[gene][transcrit][exon][0])+1 # Calcul de la longueur de l'exon en nombre de nucléotides
        total = total + int(longueur_exon) # Cette variable finit par contenir la longueur de l'exon et tous les exons précédents = position chronologique de fin pour chaque exon
        liste_position_fin_exon.append(total) # Ajoute les positions chronologiques de fin pour chaque exon

        # Conditions: la position chromosomique doit se retrouver obligatoire dans un exon du transcrit pour pouvoir être convertit en position chronologique
        if position_chromosomique >= int(gene_dic[gene][transcrit][exon][0]) and position_chromosomique <= int(gene_dic[gene][transcrit][exon][1]): 
            keys = list(gene_dic[gene][transcrit].keys()) # La variable keys renferme le nom de l'exon présentemment itéré, donc l'exon dans laquelle la position est incluse
            position_keys = keys.index(exon) # Je veux connaitre la position de l'exon dans le transcrit

            # Pour un gène sur le brin négatif
            if brand == '-':
            # Calcul de la conversion des coordonnées chromosomiques en coordonnées chronologiques dans un transcrit
                position_chronologique = int(liste_position_fin_exon[position_keys])+1+(int(gene_dic[gene][transcrit][exon][1])-position_chromosomique) 

            # Pour un gène sur le brin positif
            if brand == '+':
            # Calcul de la conversion des coordonnées chromosomiques en coordonnées chronologiques dans un transcrit
                position_chronologique = int(liste_position_fin_exon[position_keys])+1+(position_chromosomique-int(gene_dic[gene][transcrit][exon][0]))
    return position_chronologique # La fonction retourne la position chronologique dans le transcrit

In [15]:

##### ÉTAPE 4: TRIAGE DES MUTATIONS EN FONCTION DU TYPE #####
# À la fin de cette étape, toutes les mutations qui s'appliquent à un gène sont triés selon des catégories et les mutations se retrouvent
# classées dans des fichiers .tsv

# Cette première section sert à supprimer le contenu existant déjà dans le fichier si nécessaire 
#mutation = ['del_ins', 'deletion', 'duplication', 'insertion', 'substitution', 'chevauchement', 'rejected', 'autre', 'unknow', 'inversion'] # Cette liste contient tous les types de mutations
#for element in mutation: # Itération sur chaque élément présent dans la liste
    #nom_fichier = f"{gene}_mutation_{element}.tsv"
    #with open(nom_fichier, mode='w') as file: # Ouverture du fichier en mode write()
        #file.write('') # Supprime le contenu du fichier

# Avec toutes les for loop réalisés dans ce scripte, il faut s'assurer que les lignes des mutations sont ajoutés une seule fois maximum dans
# chaque fichier, sinon la même mutation sera réalisée plusieurs fois sur tous les transcrits = doublons. Le set ligne_ajoutee d'empêcher qu'
# une ligne soit ajoutée plus d'une fois

ligne_ajoutee = set()
data = 'Cosmic_MutantCensus_v101_GRCh38.tsv'
with open(data, 'r') as file: # Ouverture du fichier qui contient toutes les mutations pour tous les gènes
    for line in file: # Itération sur chaque ligne du fichier
        colonne = line.split('\t') # Séparation dans chaque ligne à l'aide des caractère \t
        if colonne[0]== gene_name: # Conserve les lignes avec le nom commun du gène, donc les mutations pour le gène d'intérêt
            mutation_type = colonne[11] 
            transcrit_reference_info = re.search(r'ENST[1234567890]+', colonne[2]) # Recherche le nom du transcrit de référence ENST
            transcrit_reference = transcrit_reference_info.group() # Stock le nom du transcrit de référence dans une variable
            mutation_grand = colonne[16] # Stock la plus grande position CHROMOSOMIQUE de la mutation
            mutation_petit = colonne[15] # Stock la plus petite position CHROMOSOMIQUE de la mutation
            mutation_explain = colonne[9] # Description de la mutation 
            mutation_id = colonne[8] # Identifiant de la mutation
            mutation_descrip = colonne[22] # Autre descriptif de la mutation



            # Gérer les mutations Unknow: elles sont retirés puisqu'elles ne possèdent pas assez d'information pour être exécutée
            if mutation_grand == '' or mutation_petit == '':
                nom_fichier = f"{gene}_mutation_unknow.tsv"
                with open(nom_fichier, mode='a') as unknow: # Ouverture du fichier en mode append()
                    unknow.write(line) # Écriture de la mutation dans le fichier

            else:
                # Il faut séparer les mutations qui tombent dans un transcrit, dans les introns ou qui chevauchent les exons et les introns (souvent des splice_donor ou splice_acceptor)

                for exon in gene_dic[gene][transcrit_reference]: # Itération sur chaque exon faisant partie du transcrit de référence

                    # La mutation doit tomber dans un exon du transcrit de référence! 
                    # Contrôle que la ligne est ajoutée une seule fois dans le fichier: son mutation_ID ne doit pas se retrouver dans le set()! Dès que la ligne est ajoutée une fois dans le fichier, son mutation_ID est ajouté en même temps dans le set()
                    if ((int(mutation_grand) >= int(gene_dic[gene][transcrit_reference][exon][0]) and int(mutation_grand) <= int(gene_dic[gene][transcrit_reference][exon][1])) and (int(mutation_petit) >= int(gene_dic[gene][transcrit_reference][exon][0]) and int(mutation_petit) <= int(gene_dic[gene][transcrit_reference][exon][1]))) and not mutation_id in ligne_ajoutee:

                        # Séparation des types de mutations. Les types de mutations sont triés dans des fichiers .tsv 

                        # Si la mutation est une substitution:
                        if '>' in mutation_explain:
                            nom_fichier = f"{gene}_mutation_substitution.tsv"
                            with open(nom_fichier, mode='a') as f: # Ouverture du nouveau fichier en mode append()
                                f.write(line) # Écriture dans le fichier
                                ligne_ajoutee.add(mutation_id) # Marquage

                        # Si la mutation est une délétion:    
                        elif mutation_explain.endswith('del'):
                            nom_fichier = f"{gene}_mutation_deletion.tsv"
                            with open(nom_fichier, mode='a') as fichier: # Ouverture du nouveau fichier en mode append()
                                fichier.write(line) # Écriture dans le fichier
                                ligne_ajoutee.add(mutation_id) # Marquage

                        # Si la mutation est une duplication:
                        elif mutation_explain.endswith('dup'):

                            # Duplication de plusieurs nucléotides
                            if '_' in mutation_descrip:
                                match = re.search(r'g\.(\d+)_([\d]+)', mutation_explain) # Recherche la position chromosomique de départ et de fin des nucléotides dupliqués
                                if match: # Si un match est trouvé
                                    small_pos = int(match.group(1)) # Stock la plus petite position chromosomique 
                                    big_pos = int(match.group(2)) # Stock la plus grande position chromosomique
                                    for exon in gene_dic[gene][transcrit_reference]:
                                        if (small_pos >= int(gene_dic[gene][transcrit_reference][exon][0]) and small_pos <= int(gene_dic[gene][transcrit_reference][exon][1])) and (big_pos >= int(gene_dic[gene][transcrit_reference][exon][0]) and big_pos <= int(gene_dic[gene][transcrit_reference][exon][1])) and not mutation_id in ligne_ajoutee:
                                            nom_fichier = f"{gene}_mutation_duplication.tsv"
                                            with open(nom_fichier, mode='a') as document: # Ouverture du nouveau fichier en mode append()
                                                document.write(line) # Écriture dans le fichier
                                                ligne_ajoutee.add(mutation_id) # Marquage
                                        else:
                                            if not mutation_id in ligne_ajoutee:
                                                nom_fichier = f"{gene}_mutation_chevauchement.tsv"
                                                with open(nom_fichier, mode='a') as document: # Ouverture du nouveau fichier en mode append()
                                                    document.write(line) # Écriture dans le fichier
                                                    ligne_ajoutee.add(mutation_id) # Marquage
                            # Duplication d'un nucléotide
                            elif not '_' in mutation_descrip:
                                nom_fichier = f"{gene}_mutation_duplication.tsv"
                                with open(nom_fichier, mode='a') as document: # Ouverture du nouveau fichier en mode append()
                                    document.write(line) # Écriture dans le fichier
                                    ligne_ajoutee.add(mutation_id)
                    
                        # Si la mutation est une insertion:
                        elif 'ins' in mutation_explain and not 'delins' in mutation_explain:
                            nom_fichier = f"{gene}_mutation_insertion.tsv"
                            with open(nom_fichier, mode='a') as papier: # Ouverture du nouveau fichier en mode append()
                                papier.write(line) # Écriture dans le fichier
                                ligne_ajoutee.add(mutation_id) # Marquage

                        # Si la mutation est une délétion-insertion:
                        elif 'delins' in mutation_explain:
                            nom_fichier = f"{gene}_mutation_del_ins.tsv"
                            with open(nom_fichier, mode='a') as tableau: # Ouverture du nouveau fichier en mode append()
                                tableau.write(line) # Écriture dans le fichier
                                ligne_ajoutee.add(mutation_id) # Marquage

                        # Si la mutation est une inversion:
                        elif mutation_explain.endswith('inv'):
                            nom_fichier = f"{gene}_mutation_inversion.tsv"
                            with open(nom_fichier, mode='a') as fich: # Ouverture du nouveau fichier en mode append()
                                fich.write(line) # Écriture dans le fichier
                                ligne_ajoutee.add(mutation_id) # Marquage
                    
                        # Si la mutation n'entre pas dans une des catégories précédentes. En principe, si toutes les catégories ont été définies, ce fichier devrait être vide ou inexistant
                        else:
                            if not mutation_id in ligne_ajoutee:
                                nom_fichier = f"{gene}_mutation_autre.tsv"
                                with open(nom_fichier, mode='a') as paper: # Ouverture du nouveau fichier en mode append()
                                    paper.write(line) # Écriture dans le fichier
                                    ligne_ajoutee.add(mutation_id) # Marquage
    
                    # Si la mutation se retrouve dans un chevauchement exon/intron, l'une de ces coordonnées chromosomiques ne fera pas partie d'un exon
                    else:
                        if ((int(mutation_grand) >= int(gene_dic[gene][transcrit_reference][exon][0]) and int(mutation_grand) <= int(gene_dic[gene][transcrit_reference][exon][1])) or (int(mutation_petit) >= int(gene_dic[gene][transcrit_reference][exon][0]) and int(mutation_petit) <= int(gene_dic[gene][transcrit_reference][exon][1]))) and not mutation_id in ligne_ajoutee:
                            nom_fichier = f"{gene}_mutation_chevauchement.tsv"
                            with open(nom_fichier, mode='a') as f: # Ouverture du nouveau fichier en mode append()
                                f.write(line) # Écriture dans le fichier
                                ligne_ajoutee.add(mutation_id) # Marquage
                # Toutes les lignes restantes: mutation dans les introns REJETÉE
                else:
                    if not mutation_id in ligne_ajoutee: # Retient les lignes qui n'ont pas été ajouté seulement
                        nom_fichier = f"{gene}_mutation_rejected.tsv"
                        with open(nom_fichier, mode='a') as fichier: # Ouverture du nouveau fichier en mode append()
                            fichier.write(line) # Écriture dans le fichier
                            ligne_ajoutee.add(mutation_id)
            if mutation_id not in ligne_ajoutee:
                nom_fichier = f'{gene}_mutation_wtf.tsv'
                with open(nom_fichier, mode='a') as fichier:
                    fichier.write(line)
                    ligne_ajoutee.add(mutation_id)
                    



In [16]:
##### ÉTAPE 5: RÉPARTIR LES MUTATIONS POUR CHAQUE TRANSCRIT #####
# Lorsqu'une mutation tombe dans un exon contenu dans un transcrit, la ligne portant sur la mutation est ajouté dans un fichier .tsv qui
# contiendra toutes les mutations qui s'appliquent à un transcrit spécifique

# Cette partie sert à effacer le contenu déjà présent dans le fichier si nécessaire
#for transcrit in gene_dic[gene]: # Itération sur chaque transcrit présent dans le gène
    #nom_fichier = f"{gene}_{transcrit}_mutations.tsv"
    #with open(nom_fichier, mode='w') as file: # Ouverture du fichier en mode append()
        #file.write('') # Suppression du contenu à l'intérieur du fichier
        #file.close() # Fermeture du fichier

# Faire une liste avec toutes les catégories de mutations à traiter
traited = ['duplication', 'substitution', 'insertion', 'deletion', 'inversion', 'del_ins'] #'chevauchement', 'rejected', 'autre' et 'unknow' ne sont pas traité dans cette partie
for item in traited: # Itération sur chaque item contenu dans la liste précédente
    try:
        with open(f'{gene}_mutation_{item}.tsv', 'r') as file: # Ouvrir le fichier contenant les mutations triées
            for line in file: # Itération sur chaque ligne du fichier
                colonne = line.split('\t') # Séparation dans les lignes à l'aide des caractères \t
                mutation_petite = int(colonne[15]) # Stockage de la plus petite position chromosomique de la mutation
                mutation_grande = int(colonne[16]) # Stockage de la plus grande position chromosomique de la mutation
                mutation_id = colonne[8]
                for transcrit in gene_dic[gene]: # Itération sur chaque transcrit du gène
                    for exon in gene_dic[gene][transcrit]: # Itération sur chaque exon d'un transcrit
                        #La mutation doit se trouver dans un exon du transcrit et elle sera ajoutée au fichier tsv
                        if (mutation_petite >= int(gene_dic[gene][transcrit][exon][0]) and mutation_petite <= int(gene_dic[gene][transcrit][exon][1])) and (mutation_grande >= int(gene_dic[gene][transcrit][exon][0]) and mutation_grande <= int(gene_dic[gene][transcrit][exon][1])):
                            nom_fichier = f"{gene}_{transcrit}_mutations.tsv"
                            with open(nom_fichier, mode='a') as f: # Ouverture du fichier en mode append(). Ce fichier va contenir toutes les mutations de n'importe quel type qui s'appliquent à un transcrit spécifique
                                f.write(line) # Écriture de la ligne dans le fichier

    except FileNotFoundError:
        # Si le fichier n'existe pas, ne rien faire et passer au suivant. Cela peut être possible dans le cas où un gène ne contient pas tous les types de mutations existant
        continue


In [17]:
##### ÉTAPE 6: PRODUIRE CHAQUE TRANSCRIT AVEC CHAQUE MUTATION APPLICABLE #####
# Cette section produit toutes les séquences d'ADNc des transcrits avec chaque mutation de n'importe quelle catégorie qui sont applicables.
# Les séquences mutés sont ajoutés dans des fichiers fasta triés en fonction du type de mutation effectué 


# Cette partie sert à effacer le contenu des fichiers fasta déjà existant
for item in traited:
    #nom_fichier = f"{gene}_{item}.fasta"
    #with open(nom_fichier, mode='w') as file: # Ouverture du fichier en mode write()
        #file.write('') # Supprime le contenu du fichier
    nom_fichier = f"{item}_not_working.tsv"
    with open(nom_fichier, mode='w') as file: # Ouverture du fichier en mode write()
        file.write('') # Supprime le contenu du fichier


lignes_deja_ajoutees = set() # Le set() contiendra, une seule fois, toutes les mutations (de n'importe quelle catégorie) qui ne fonctionnent pas
for transcrit in gene_dic[gene]: # Itération sur chaque transcrit du gène

    sequence = sequence_cDNA(transcrit) # Obtention de la séquence ADNc d'un transcrit spécifique
    header1 = '>' + f'{gene}_{transcrit}' # Formation du header
    fasta = f'{gene}_transcrits_no_mutation.fasta'
    with open(fasta, 'a') as f: # Ouverture du fichier fasta
        f.write(header1 + '\n') # Écriture du header
        f.write(sequence + '\n') # Écriture de la séquence
        f.close()

    data = f'{gene}_{transcrit}_mutations.tsv'
    try:
        with open(data, 'r') as file: # Ouverture du fichier du transcrit contenant toutes ses mutations pour toutes les catégories confondues

            for line in file: # Itération sur chaque ligne du fichier, donc sur chaque mutation
                colonne = line.split('\t') # Séparation dans les lignes à l'aide des caractère \t
                changement = colonne[9] # Description de la mutation
                position_petite = int(colonne[15]) # Stockage de la plus petite position chromosomique de la mutation
                position_grande = int(colonne[16]) # Stockage de la plus grande position chromosomique de la mutation
                mutation_explain = colonne[22] # Autre colonne qui décrit la mutation, elle est surtout utile dans le cas des duplications
                mutation_id = colonne[8] # Identifiant de la mutation

                # Conversion des positions chromosomiques de la mutation en position chronologique dans le transcrit COMMENÇANT PAR 0
                position_chrono_1 = (get_position_chronologique(position_petite, transcrit))-1 
                position_chrono_2 = (get_position_chronologique(position_grande, transcrit))-1

                # Créer la liste suivante permet de pouvoir ressortir la position de départ et de fin de la mutation peut importe si la mutation 
                # est sur le brin négatif ou positif (EX. pour le brin +, la position de départ provient de la position chromosomique la plus faible,
                # mais pour le brin -, elle provient de la position chromosomique la plus grande)
                liste = [position_chrono_1, position_chrono_2] # Ajoute les deux positions chronologiques dans la liste
                brin = colonne[17] # Stock la nature du brin +/-

                if brin == '+':
                    delete = colonne[23] # Stock la séquence qui est retirée
                    add = colonne[24] # Stock la séquence qui est ajoutée

                if brin == '-': # Si le gène est sur le brin -, la database donne la séquence du brin inverse et complémentaire
                    enleve = colonne[23]
                    chaine_inverse = enleve[::-1] # Inversion (miroir) de la chaine
                    # Obtention de la séquence complémentaire 
                    adn1 = chaine_inverse.replace('A', 't') 
                    adn2 = adn1.replace('T', 'a')
                    adn3 = adn2.replace('C', 'g')
                    adn4 = adn3.replace('G', 'c')
                    delete = adn4.upper() # Change les caractères en majuscule

                    ajoute = colonne[24]
                    chaine_miroir = ajoute[::-1] # Inversion (miroir) de la chaine
                    # Obtention de la séquence complémentaire
                    adn5 = chaine_miroir.replace('A', 't')
                    adn6 = adn5.replace('T', 'a')
                    adn7 = adn6.replace('G', 'c')
                    adn8 = adn7.replace('C', 'g')
                    add = adn8.upper()
                    
                if position_petite == position_grande and '>' in changement: # Isole les substitutions
                    position_chrono = (get_position_chronologique(position_petite, transcrit))-1 # Calcul de la position chronologique de la mutation en commençant par 0
                    base_avant = changement[-3] # Stockage de la base initialement contenu dans le transcrit
                    base_apres = changement[-1] # Stockage de la base qui va venir remplacée l'ancienne base
                    if sequence[position_chrono]== base_avant: # Étape de vérification: si on veut effectuer la substitution, alors il faut que la base qui trouve à la position changée corresponde à la base avant
                        sequence_modifiee = sequence[:position_chrono]+ base_apres + sequence[position_chrono+1:] # Formation de la séquence mutée
                        header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header
                        fasta = f'{gene}_substitution.fasta'
                        with open(fasta, 'a') as f: # Ouverture du fichier fasta qui contiendra tous les transcrits mutés avec des substitutions
                            f.write(header + '\n') # Écriture du header
                            f.write(sequence_modifiee + '\n') # Écriture de la séquence

                    # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                    else: 
                        if line not in lignes_deja_ajoutees: # Cette ligne permet d'ajouter une seule fois chaque mutation au fichier qui récupère les substitutions défectueuses. Pour ajouter la ligne, elle ne doit pas être dans le set()
                            reste = 'substitution_not_working.tsv'
                            with open(data, 'r') as r: # Ouverture du fichier qui va contenir les substitutions défectueuses en mode append()
                                r.write(line) # Écriture de la ligne dans le fichier
                                lignes_deja_ajoutees.add(line) # Ajoute la ligne dans le set(), ce qui fait qu'elle ne pourra pas être ajouté dans le fichier plus d'une fois

            
                # GÉRER LES DELETIONS
                elif changement.endswith('del'): # Cette ligne permet de retenir toutes les mutations de types délétions
                    smallest = min(liste) # Stock la plus petite position chronologique = DÉBUT DE LA MUTATION
                    biggest = max(liste) # Stock la plus grande position chronologique = FIN DE LA MUTATION
                    remove_sequence = sequence[smallest:biggest+1] # Stock la séquence deletée dans l'ADNc
                    if remove_sequence == delete: # Étape de vérification, la séquence retirée dite dans la database de COSMIC doit être identique à la séquence retirée dans l'ADNc
                        sequence_modifiee = sequence[:smallest] + sequence[biggest+1:] # Formation de la séquence modifiée
                        header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header à l'aide du gène, du transcrit et de l'ID de la mutation
                        fasta = f'{gene}_deletion.fasta' 
                        with open (fasta, 'a') as f: # Ouverture du fichier fasta qui va contenir les séquences des transcrits mutés
                            f.write(header + '\n') # Écriture du header
                            f.write(sequence_modifiee + '\n') # Écriture de la séquence
                    # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                    else: 
                        if line not in lignes_deja_ajoutees: # Cette ligne permet d'ajouter une seule fois chaque mutation au fichier qui récupère les deletions défectueuses. Pour ajouter la ligne, elle ne doit pas se retrouver dans le set()
                            reste = 'deletion_not_working.tsv'
                            with open(reste, 'a') as r: # Ouverture du fichier qui récupère les deletions défectueuses en mode append()
                                r.write(line) # Écriture de la ligne dans le fichier
                                lignes_deja_ajoutees.add(line) # Ajoute la ligne dans le set(), ce qui fait qu'elle ne pourra pas être ajouté dans le fichier plus d'une fois

                # GÉRER LES INSERTIONS
                elif 'ins' in changement and not 'delins' in changement: # Cette ligne permet de conserver les lignes qui portent sur les mutations de type insertion
                    smallest = min(liste) # Stock la plus petite position chronologique = position de départ de la mutation
                    biggest = max(liste) # Stock la plus grande position chronologique = position de fin de la mutation
                    pattern = r'[ATCG]+' # Identification du pattern, le pattern correspond à la séquence d'ADN inséré dans l'ADNc
                    match = re.search(pattern, changement) # Recherche le pattern à l'aide des regular expression dans la colonne qui décrit la mutation
                    sequence_ajoutee = match.group() # Stock la séquence ajoutée dans une variable
                    if sequence_ajoutee == add: # Étape de vérification: s'assurer que la séquence ajoutée établit aux 2 endroits du fichier .tsv COSMICS sont identiques
                        sequence_modifiee = sequence[:biggest] + sequence_ajoutee + sequence[biggest:] # Formation de la séquence mutés
                        header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header à l'aide du gène, du transcrit et de l'ID de la mutation
                        fasta = f'{gene}_insertion.fasta'
                        with open (fasta, 'a') as f: # Ouverture du fichier fasta en mode append() qui va contenir tous les transcrits mutés avec les mutations insertions
                            f.write(header + '\n') # Écriture du header
                            f.write(sequence_modifiee + '\n') # Écriture de la séquence
                    # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                    else: 
                        if line not in lignes_deja_ajoutees: # Cette ligne permet d'ajouter une seule fois la ligne de la mutation défectueuse au fichier
                            reste = 'insertion_not_working.tsv'
                            with open(reste, 'a') as r: # Ouverture du fichier qui va contenir les mutations insertions défectueuses
                                r.write(line) # Écriture de la ligne de la mutation
                                lignes_deja_ajoutees.add(line) # Ajout de la ligne dans le set()

                # GÉRER LES DUPLICATIONS
                elif changement.endswith('dup'): # Cette ligne permet de conserver les mutations de type duplication
                # Le scripte est différent si on a une duplication d'un nucléotide ou de plusieurs nucléotides

                    if '_' in mutation_explain: # Cette ligne permet de conserver les duplications de plusieurs nucléotides
                        match = re.search(r'g\.(\d+)_([\d]+)', mutation_explain) # Recherche la position chromosomique de départ et de fin des nucléotides dupliqués
                        if match: # Si un match est trouvé
                            small_pos = int(match.group(1)) # Stock la plus petite position chromosomique 
                            big_pos = int(match.group(2)) # Stock la plus grande position chromosomique
                            # Conversion des positions chromosomiques en position chronologique dans le transcrit en commençant par 0
                            chrono_1 = (get_position_chronologique(small_pos, transcrit)-1)
                            chrono_2 = (get_position_chronologique(big_pos, transcrit)-1)
                            liste_position_chrono = [chrono_1, chrono_2] # Ajoute les positions chronologiques dans la liste
                            # La liste précédente permet de ressortir les positions de départ et de fin de mutation peut importe si le gène est sur le brin + ou -.
                            biggest = max(liste_position_chrono) # Stock la plus grande position chronologique: fin de la mutation
                            smallest = min(liste_position_chrono) # Stock la plus petite position chronologique: début de la mutation
                            sequence_dupliquee = sequence[smallest: biggest+1] # Stock les nucléotides qui sont dupliqués
                            if sequence_dupliquee == add: # Étape de vérification: les nucléotides retrouvés dans la séquence d'ADNc doivent être identiques aux nucléotides qui seront dupliqués selon COSMICS
                                sequence_modifiee = sequence[:biggest+1] + sequence_dupliquee + sequence[biggest+1:] # Formation de la séquence mutées
                                header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header
                                fasta = f'{gene}_duplication.fasta'
                                with open (fasta, mode = 'a') as f: # Ouverture du fichier fasta qui contiendra les séquences des transcrits mutés par des duplications en mode append()
                                    f.write(header + '\n') # Écriture du header
                                    f.write(sequence_modifiee + '\n') # Écriture de la séquence
                            # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                            else: 
                                if line not in lignes_deja_ajoutees: # Cette ligne permet d'ajouter une seule fois la ligne de la mutation dans le fichier qui contiendra les mutations défectueuses
                                    reste = 'duplication_not_working.tsv'
                                    with open(reste, mode = 'a') as r: # Ouverture du fichier qui va contenir les mutations duplications défectueuses en mode append()
                                        r.write(line) # Écriture de la ligne de la mutation
                                        lignes_deja_ajoutees.add(line) # Ajoute la ligne dans le set(), ce qui permet de ne pas ajouter la même ligne dans le fichier plus d'une fois
                    else:
                        if not '_' in mutation_explain: # Cette ligne permet de conserver les duplications d'un seul nucléotide
                            match = re.search(r'g\.(\d+)dup', mutation_explain) # Recherche la position chromosomique du nucléotide qui sera dupliqué
                            if match: # Si un match est trouvé
                                position = int(match.group(1)) # Stock la position chromosomique du nucléotide dupliqué dans une variable
                                chrono_position = int(get_position_chronologique(position, transcrit)-1) # Conversion de la position chromosomique en position chronologique dans le transcrit
                                sequence_dupliquee = sequence[chrono_position] # Stockage du nucléotide dupliqué
                                if sequence_dupliquee == add: # Le nucléotide dupliquée dans la séquence ADNc doit être le même que celui dupliqué selon COSMICS
                                    sequence_modifiee = sequence[:chrono_position+1] + sequence_dupliquee + sequence[chrono_position+1:] # Formation de la séquence mutée
                                    header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header
                                    fasta = f'{gene}_duplication.fasta' 
                                    with open (fasta, mode = 'a') as f: # Ouverture du fichier fasta qui va contenir tous les transcrits mutés par des duplications
                                        f.write(header + '\n') # Écriture du header
                                        f.write(sequence_modifiee + '\n') # Écriture de la séquence
                                # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                                else:
                                    if line not in lignes_deja_ajoutees: # Cette ligne permet d'ajouter une seule fois la ligne de la mutation dans le fichier qui contiendra les mutations défectueuses
                                        reste = 'duplication_not_working.tsv'
                                        with open(reste, mode = 'a') as r: # Ouverture du fichier qui va contenir les mutations défectueuses
                                            r.write(line) # Écriture de la ligne de la mutation
                                            lignes_deja_ajoutees.add(line) # Ajout de la ligne dans le set(), qui va permettre de ne pas ajouter la même ligne plus d'une fois dans le fichier

                # GÉRER LES DELETION-INSERTION
                elif 'delins' in changement: # Cette ligne conserve les mutations deletion/insertion
                    # Traitement de la séquence deletée
                    smallest = min(liste) # Stock la plus petite position chronologique = début de la mutation
                    biggest = max(liste) # Stock la plus grande position chronologique = fin de la mutation
                    remove_sequence = sequence[smallest:biggest+1] # Stock la séquence deleté

                    # Traitement de la séquence ajoutée
                    pattern = r'[ATCG]+' # Identification du pattern qui est la séquence d'ADN inséré
                    match = re.search(pattern, changement) # Recherche la séquence ajoutée
                    sequence_ajoutee = match.group() # Stock la séquence ajoutée

                    if remove_sequence == delete and sequence_ajoutee == add: # Étape de vérification
                        sequence_modifiee = sequence[:smallest] + sequence_ajoutee + sequence[biggest+1:] # Formation de la séquence mutée
                        header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header à l'aide du gène, du transcrit et de l'ID de la mutation
                        fasta = f'{gene}_del_ins.fasta'
                        with open (fasta, 'a') as f: # Ouverture du fichier fasta qui va contenir tous les transcrits mutés par des del-ins en mode append()
                            f.write(header + '\n') # Écriture du header
                            f.write(sequence_modifiee + '\n') # Écriture de la séquence
                    # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                    else: 
                        if line not in lignes_deja_ajoutees: # Cette ligne empêche que la même mutation soit ajoutée plusieurs fois dans le fichier
                            reste = 'del_ins_not_working.tsv'
                            with open(reste, mode = 'a') as r: # Ouverture du fichier qui va contenir les mutations défectueuses en mode append()
                                r.write(line) # Écriture de la ligne de la mutation
                                lignes_deja_ajoutees.add(line) # Ajout de la ligne dans le set()

                # GÉRER LES INVERSIONS
                elif changement.endswith('inv'): # Cette ligne conserve les mutations inversions
                    smallest = min(liste) # Stock la plus petite position chronologique = début de la mutation
                    biggest = max(liste) # Stock la plus grande position chronologique = fin de la mutation
                    sequence_inverse = sequence[smallest: biggest+1] # Identification des nucléotides qui seront inversés
                    if sequence_inverse == delete: # Étape de vérification
                        sequence_modifiee = sequence[:smallest] + sequence_inverse[::-1] + sequence[biggest+1:] # Formation de la séquence mutée
                        header = '>' + f'{gene}_{transcrit}_{mutation_id}' # Formation du header
                        fasta = f'{gene}_inversion.fasta' 
                        with open (fasta, 'a') as f: # Ouverture du fichier fasta qui va contenir les transcrits mutés par des inversion
                            f.write(header + '\n') # Écriture du header
                            f.write(sequence_modifiee + '\n') # Écriture du header
                    # En théorie, si aucun problème n'est rencontré, la section suivante ne devrait pas s'effectuer
                    else:
                        if line not in lignes_deja_ajoutees: # Cette ligne empêche que la même mutation soit ajoutée plusieurs fois dans le fichier
                            reste = 'inversion_not_working.tsv'
                            with open(reste, 'a') as r: # Ouverture du fichier qui va contenir les mutations inversion défectueuses en mode append()
                                r.write(line) # Écriture de la ligne de la mutation
                                lignes_deja_ajoutees.add(line) # Ajoute la ligne au set()
    except FileNotFoundError:
        continue

# Il faut conserver toutes les mutations qui ne fonctionnent pas
    
# Liste de tous les fichiers fasta voulant être combiné
fichiers_fasta = ['substitution_not_working.tsv', 'insertion_not_working.tsv', 'inversion_not_working.tsv', 'del_ins_not_working.tsv', 'deletion_not_working.tsv', 'duplication_not_working.tsv']
# Le nom du fichier qui va contenir tous les contenus
fichier_sortie = f'{gene}_not_working.tsv'

with open(fichier_sortie, 'a') as fichier_combinaison: # Ouverture du fichier qui va contenir tous les contenus
    for fichier in fichiers_fasta: # Itération sur chaque fichier contenu dans la liste
        try:
            with open(fichier, 'r') as f: # Ouverture du fichier
                fichier_combinaison.write(f.read()) # Écrit le contenu du fichier dans le gros fichier
        except FileNotFoundError:
            continue

In [18]:
##### ÉTAPE 7: ÉTAPE DE VÉRIFICATION #####
# Cette section permet de vérifier si le nombre de mutation effectuée pour chaque catégorie correspond au nombre de mutation supposé. On compare
# deux nombres ensemble et s'ils s'équivalent, cela signifie que le scripte fonctionne

fichiers_fasta = [f'{gene}_mutation_substitution.tsv', f'{gene}_mutation_insertion.tsv', f'{gene}_mutation_deletion.tsv', f'{gene}_mutation_duplication.tsv', f'{gene}_mutation_del_ins.tsv', f'{gene}_mutation_inversion.tsv']
# Le nom du fichier qui va contenir tous les contenus
fichier_sortie = f'{gene}_toutes_les_mutations_exons.tsv'
with open(fichier_sortie, 'w') as fichier_combinaison: # Ouverture du fichier qui va contenir tous les contenus
    for fichier in fichiers_fasta: # Itération sur chaque fichier contenu dans la liste
        try:
            with open(fichier, 'r') as f: # Ouverture du fichier
                fichier_combinaison.write(f.read()) # Écrit le contenu du fichier dans le gros fichier
        except FileNotFoundError:
            continue

every_fichier_fasta = [f'{gene}_mutation_substitution.tsv', f'{gene}_mutation_insertion.tsv', f'{gene}_mutation_deletion.tsv', f'{gene}_mutation_duplication.tsv', f'{gene}_mutation_del_ins.tsv', f'{gene}_mutation_inversion.tsv', f'{gene}_mutation_autre.tsv', f'{gene}_mutation_rejected.tsv', f'{gene}_mutation_chevauchement.tsv', f'{gene}_mutation_wtf.tsv']
fichier_sortie = f'{gene}_toutes_les_mutations_intron_et_exon.tsv'
with open(fichier_sortie, 'w') as fichier_combinaison: # Ouverture du fichier qui va contenir tous les contenus
    for fichier in every_fichier_fasta: # Itération sur chaque fichier contenu dans la liste
        try:
            with open(fichier, 'r') as f: # Ouverture du fichier
                fichier_combinaison.write(f.read()) # Écrit le contenu du fichier dans le gros fichier
        except FileNotFoundError:
            continue

with open (f'{gene}_toutes_les_mutations_intron_et_exon.tsv', 'r') as file: # Ouverture de chaque fichier fasta qui contient les séquences mutés pour chaque type de mutation
    count = 0 # Cette variable va contenir le nombre de ligne dans le fichier
    for line in file: # Itération sur chaque ligne du fichier
        count += 1 # Ajoute +1 à chaque ligne
                    
    
mutation_identifiant = set()
with open('Cosmic_MutantCensus_v101_GRCh38.tsv', 'r') as file:
    for line in file:
        colonne = line.split('\t')
        mutation_id = colonne[8]
        if colonne[0]== gene_name:
            if mutation_id != '':
                mutation_identifiant.add(mutation_id)
    
print( str(count) + ' should be egal to ' +  str(len(mutation_identifiant)))
    


for item in traited: # Itération sur chaque élément contenu dans la liste traited
    try:
        with open (f'{gene}_{item}.fasta', 'r') as file: # Ouverture de chaque fichier fasta qui contient les séquences mutés pour chaque type de mutation
            line_count = 0 # Cette variable va contenir le nombre de ligne dans le fichier
            for line in file: # Itération sur chaque ligne du fichier
                line_count += 1 # Ajoute +1 à chaque ligne
        compte = 0 # Contiendra le nombre de mutation du type à effectuer pour un transcrit spécifique
        for transcrit in gene_dic[gene]: # Itération sur chaque transcrit
            data = f'{gene}_mutation_{item}.tsv'
            with open(data, 'r') as fichier: # Ouverture de chaque fichier qui contient les types de mutations triés
                for line in fichier: # Itération sur chaque du fichier ouvert
                    if line not in lignes_deja_ajoutees: # On compte seulement les lignes qui ne sont pas dans le set(), car ces mutations n'ont pas été effectuée
                        colonne = line.split('\t') # Séparation dans les lignes à l'aide des caractères \t
                        mutation_end = colonne[16] # Fin de la mutation
                        mutation_start = colonne[15] # Début de la mutation
                        changement = colonne[9] # Descriptif de la mutation
                        for exon in gene_dic[gene][transcrit]: # Itération sur chaque exon du transcrit
                            # La mutation doit se retrouver dans un exon du transcrit
                            if (int(mutation_start) >= int(gene_dic[gene][transcrit][exon][0]) and int(mutation_start) <= int(gene_dic[gene][transcrit][exon][1])) and (int(mutation_end) >= int(gene_dic[gene][transcrit][exon][0]) and int(mutation_end) <= int(gene_dic[gene][transcrit][exon][1])):
                                compte = compte +1 # Ajoute +1

        # Comparaison des deux valeurs: un fichier fasta contient deux lignes pour une mutation (séquence + header), c'est pourquoi on divise par deux
        if line_count/2 == compte: 
            print(f'{item}' + ' is correct! ' + str(line_count) + '/2 = ' + str(compte)) 
        else:
            print(f'Error for {item}')

    except FileNotFoundError:
        # Si le fichier n'existe pas, ne rien faire et passer au suivant
        continue
    

with open (f'{gene}_transcrits_no_mutation.fasta', 'r') as file: # Ouverture de chaque fichier fasta qui contient les séquences mutés pour chaque type de mutation
    compte_ligne = 0 # Cette variable va contenir le nombre de ligne dans le fichier
    for line in file: # Itération sur chaque ligne du fichier
        compte_ligne += 1 # Ajoute +1 à chaque ligne

if compte_ligne/2 == len(gene_dic[gene]):
    print('Le nombre de transcrit correspond à celui dans le gene_dic!')
else:
    print('Error in the number of transcripts')

                   

1969 should be egal to 1969
duplication is correct! 834/2 = 417
substitution is correct! 57002/2 = 28501
insertion is correct! 570/2 = 285
deletion is correct! 4622/2 = 2311
del_ins is correct! 918/2 = 459
Le nombre de transcrit correspond à celui dans le gene_dic!


In [19]:
##### ÉTAPE 8: AJOUTER TOUS LES TRANSCRITS MUTÉS DANS UN GROS FASTA #####
# Cette section permet de combiner tous les sous-fichier fasta qui contiennent les séquences mutés et non-muté classées selon chaque type de mutation, dans
# un seul gros fichier fasta.

# Cette partie sert à effacer le contenu du fichier s'il existe déjà
#nom_fichier = f'{gene}_all_transcripts_with_mutation.fasta'
#with open(nom_fichier, mode='w') as file: # Ouverture du fichier en mode write()
    #file.write('') # Suppression du contenu
    
# Liste de tous les fichiers fasta voulant être combiné
fichiers_fasta = [f'{gene}_substitution.fasta', f'{gene}_insertion.fasta', f'{gene}_deletion.fasta', f'{gene}_duplication.fasta', f'{gene}_del_ins.fasta', f'{gene}_inversion.fasta', f'{gene}_transcrits_no_mutation.fasta']
# Le nom du fichier qui va contenir tous les contenus
fichier_sortie = f'{gene}_all_transcripts_with_mutation.fasta'

with open(fichier_sortie, 'w') as fichier_combinaison: # Ouverture du fichier qui va contenir tous les contenus
    for fichier in fichiers_fasta: # Itération sur chaque fichier contenu dans la liste
        try:
            with open(fichier, 'r') as f: # Ouverture du fichier
                fichier_combinaison.write(f.read()) # Écrit le contenu du fichier dans le gros fichier
        except FileNotFoundError:
            continue

In [ ]:
  ##### ÉTAPE 9: DÉPLACER TOUS LES FILES SUR LE GÈNE D'INTÉRÊT DANS UN AUTRE DOSSIER ####

# Importation des modules permettant de jouer avec les fichiers
import os
import shutil

current_directory = os.getcwd() # Récupère le répertoire courant de travail (dossier où le script est exécuté)
destination_directory = '/mnt/c/Users/bedar/Documents/Documents/Cosmics_transcript_mutated' # Dossier de destination dans lequel un nouveau dossier sera créé

# Création d'un nouveau dossier dans le dossier de destination
new_folder = os.path.join(destination_directory, f'{gene}_files')
if not os.path.exists(new_folder): # Si le dossier n'existe pas, il faut en créer un autre
    os.makedirs(new_folder)

# Parcours tous les fichiers dans le répertoire courant
for filename in os.listdir(current_directory):
    if filename.startswith(f'{gene}'): # Il faut que le fichier commence par le nom du gène
        # Crée le chemin complet pour le fichier source et la destination
        source_file = os.path.join(current_directory, filename)
        destination_file = os.path.join(new_folder, filename)
        # Déplace le fichier vers le nouveau dossier
        shutil.move(source_file, destination_file)